# Explanation

The primary goal of robotic perception is to enable to the robot to build an internal map of its environment, understand the surrounding objects, and understand its position in the environment. Having such a model allows the robot to plan where to go and what to do to accomplish its goal.

Simultaneous localization and mapping (SLAM) allows the robot to build an accurate model of the world through periodic updating of a probabilistic world model after subsequent observations.

Because robotic sensors are prone to error, world models quickly accumulate inaccuracies if sensor data is taken at face value to build world model. To accommodate for this, SLAM uses overlapping information from multiple data points to resolve errors over time. Because the same data points can be observed in multiple frames from multiple positions, correlations between data points over time make it possible to reduce error and deduce accurate relative positions with more information. This also simultaneously improves the robots understanding of it's own position in the environment.



# Notes

> The simultaneous localization and mapping (SLAM) problem asks if it is possible for a mobile robot to be placed at an unknown location in an unknown environment and for the robot to incrementally build a consistent map of this environment while simultaneously determining its location within this map.

A robot has to build a map of the world and figure out where it is in the map when dropped in any new environment.

> A solution to the SLAM problem has been seen as a “holy grail” for the mobile robotics community as it would provide the means to make a robot truly autonomous.

Good SLAM is essential for an actually autonomous/intelligent humanoid robot.

> Substantial issues remain in practically realizing more general SLAM solutions and notably in building and using perceptually rich maps as part of a SLAM algorithm.

SLAM is a theoretically “solved” problem but still has room for improvement in practice.

### History of the SLAM Problem

> There must be a high degree of correlation between estimates of the location of different landmarks in a map and that, indeed, these correlations would grow with successive observations.

Estimates of different landmarks would all result from the same prediction errors on the robot.

Early work in visual navigation and sonar-based navigation with Kalman filters led to initial progress.

> As a mobile robot moves through an unknown environment taking relative observations of landmarks, the estimates of these landmarks are all necessarily correlated with each other because of the common error in estimated vehicle location.

All the landmark locations should be correlated because of the same prediction error. This can be used to update the map over time.

> A consistent full solution to the combined localization and mapping problem would require a joint state composed of the vehicle pose and every landmark position, to be updated following each landmark observation.

A SLAM solution would involve maintaining the state of every single observed landmark (point) and updating this state after each new observation.

Because of this, computation scales with the square of the number of landmarks.

> The conceptual breakthrough came with the realization that the combined mapping and localization problem, once formulated as a single estimation problem, was actually convergent.

People realized that the problem converges if you do mapping and localization together, since mapping makes the localization problem converge.

> Most importantly, it was recognized that the correlations between landmarks, which most researchers had tried to minimize, were actually the critical part of the problem and that, on the contrary, the more these correlations grew, the better the solution.

It wasn’t initially obvious that the problem converged. However, they eventually realized that as correlations between landmarks grow with new observations, the problem converges toward a solution.

### Formulation and Structure of the SLAM Problem

**1. Preliminaries**

SLAM lets a robot build a map of the world and deduce it’s location, while estimating the location of landmarks and trajectory of the robot without any prior knowledge.

We define the following sets of information for the SLAM problem

- $X_{0:k} = \{ x_0, x_1, …, x_k \}$ - the history of all robot locations
- $U_{0:k} = \{ u_0, u_1, …, u_k \}$ - the history of control inputs
- $m = \{ m_0, m_1, …, m_k\}$ - the set of all landmark locations
- $Z_{0:k} = \{ z_1, z_2, …, z_k \}$ - the set of all landmark observations

**2. Probabilistic SLAM**

The SLAM problem requires the following probability distribution to be computed for all times $k$:

$$
P(x_k, m|Z_{0:k}, U_{0:k}, x_0)
$$

Given all prior landmark observations $Z_{0:k}$ and control inputs $U_{0:k}$, and the starting position $x_0$ of the robot as a reference point, the robot has to compute it’s own current position $x_k$ in the environment and the locations of all landmarks $m$ as a solution to the SLAM problem.

We ideally want a recursive solution that starts with the information and prediction from the prior time step:

$$
P(x_{k-1}, m|Z_{0:k-1}, U_{0:k-1})
$$

And uses the new control $u_k$ and observation $z_k$ to compute the new distribution.

The **observation model** describes the probability of making an observation given the map and vehicle position:

$$
P(z_k|x_k, m)
$$

The **motion model** describes the probability of the robot location given prior location and control inputs:

$$
P(x_k|x_{k-1}, u_k)
$$

The SLAM algorithm uses a two step process.

First, it makes the **time update** which updates the prediction of the position and map given all prior landmark observations $Z_{0:{k-1}}$ and all motion commands including the most recent command $U_{0:k}$:

$$
P(x_k, m|Z_{0:k-1}, U_{0:k}, x_0) = \int P(x_k| x_{k-1}, u_k) \times P(x_{k-1}, m|Z_{0:k-1}, U_{0:k-1}, x_0) \: dx_{k-1}
$$

Here, we simply predict the new robot location $x_k$, given our prior beliefs about landmark positions before moving $P(x_{k-1}, m|Z_{0:k-1}, U_{0:k-1}, x_0)$, and our prediction about where we have moved given control inputs (the motion model) $P(x_k| x_{k-1}, u_k)$.

By multiplying these distributions here, we allow the uncertainty about position $x_{k-1}$ from the previous step to influence our prediction about $x_k$, in addition to the new motion command $u_k$.

We use the entire prior distribution since our prediction about $x_{k-1}$ (and the implicitly related landmark predictions $m$) depend on all the priors, so our new prediction of $x_k$ does as well since it requires $x_{k-1}$.

Through this calculation, we recursively maintain all the relationships of our prediction $x_k$ on all the prior information influencing it.

Notably, $P(x_k, m)$ must be a joint distribution since our predictions about the robot position and map are directly related since they are naturally relative to each other. Uncertainty about the robot position increase our uncertainty about where landmarks are relative to the robot.

After the time update, we use the new predictions to make the **measurement update** which is what corrects for prediction error:

$$
P(x_k, m|Z_{0:k}, U_{0:k}, x_0) = \frac{P(z_k|x_k,m)P(x_k, m|Z_{0:k-1}, U_{0:k}, x_0)}{P(z_k|Z_{0:k-1},U_{0:k})}
$$

Specifically, we use the newly updated joint distribution (with updated position prediction) from the time-update $P(x_k, m|Z_{0:k-1}, U_{0:k}, x_0)$, and we merge this (using Bayes’ Rule) with the distribution over robot positions $x_k$ and landmark positions $m$ that would make $z_k$ most likely (the observation model) $P(z_k|x_k, m)$.

This new input from the observation model encodes new information about the correlations of robot and map positions which gets factored into our full joint distribution.

Then, we normalize by $P(z_k|Z_{0:k-1}, U_{0:k})$ to keep this as a probability distribution.

In this entire process, we see that the joint distribution is recursively integrating information from the motion model $P(x_k| x_{k-1}, u_k)$ and observation model $P(z_k|x_k, m)$.

We can view the map building problem as computing the distribution $P(m|X_{0:k}, Z_{0:k}, U_{0:k})$, and we can view the localization problem as computing the distribution $P(x_i|Z_{0:k}, U_{0:k}, m)$.

The dependence of these predictions on each other is reflected in the joint distribution.

### Structure of Probabilistic SLAM

![Screenshot 2024-11-07 at 11.50.40 AM.png](../../images/Screenshot_2024-11-07_at_11.50.40_AM.png)

> It can be seen that much of the error between estimated and true landmark locations is common between landmarks and is in fact due to a single source; errors in knowledge of where the robot is when landmark observations are made.

Most of the error accumulated over time in robotic mapping comes from errors in knowledge about the robots position.

This means that the relative locations between landmarks is generally known, but just the absolute location has error due to incorrect beliefs about the robot position.

> The most important insight in SLAM was to realize that the correlations between landmark estimates increase monotonically as more and more observations are made.

This means that predictions about landmark positions $P(m)$ always improves over time (becomes more concentrated).

> The relative location of observed landmarks is clearly independent of the coordinate frame of the vehicle, and successive observations from this fixed location would yield further independent measurements of the relative relationship between landmarks.

![Screenshot 2024-11-07 at 11.57.54 AM.png](../../images/Screenshot_2024-11-07_at_11.57.54_AM.png)

> This process can be visualized (Figure 2) as a network of
> springs connecting all landmarks together

> As the robot moves through this environment and takes observations of the landmarks, the springs become increasingly (and monotonically) stiffer.

We can think of this as increasing the strength of correlations between points as certainty increases, causing points to further update our beliefs about where they are.

> In the limit, a rigid map of landmarks or an accurate relative map of the environment is obtained.

As our map improves, are certainty about the robot position also improves.

### Solutions to the SLAM Problem

> Solutions to the probabilistic SLAM problem involve finding an appropriate representation for both the observation model and motion model that allows efficient and consistent computation of the prior and posterior distributions.

Solving the SLAM problem is about finding a good representation for the observation model and motion model, which allows us to accurately calculate the time update and measurement update.

All the SLAM solutions like EKF-SLAM, FastSLAM, etc. are all different ways to represent the distributions of the observation and motion model.

**1. EKF-SLAM**

EKF-SLAM represents the motion and observation models with Gaussian noise.

Specifically, it describes the vehicle motion as:

$$
P(x_i|x_{k-1}, u_k) \Longleftrightarrow x_k = f(x_{k-1}, u_k) + w_k
$$

Adding the prediction in a perfect scenario modeled by the expected vehicle kinematics $f(x_{k-1}, u_k)$ with zero mean Gaussian noise $w_k$ with covariance $Q_k$ to model motion noise.

Similarly, the observation model is described as:

$$
P(z_k|x_k, m) \Longleftrightarrow z_k = h(x_k, m) + v_k
$$

Adding the expected geometry of the observation $h(\cdot)$ with zero mean Gaussian noise $v_k$ with covariance $R_k$ to model observation errors.

The output of these distributions is a Gaussian distribution with some variance centered around the expected robot position and expected observations.

This distribution is re-centered over time given new observations and motion inputs, and the noise should converge to lower variance over time, leading to a higher certainty SLAM prediction.

Both of these use Extended Kalman Filters (EKF). They treat the distributions as if error can be linearized around the central point (using the $w_k$ and $v_k$ Gaussian noise, which is characteristic of a Kalman Filter.

Such a setup can only handle linear systems, but they make it compatible with non-linearities by introducing the functions $f(\cdot)$ and $h(\cdot)$ which compute an individual point to linearize the prediction around at each point in time.

This method then uses the standard EKF procedure to update the mean and covariance matrices over time through calculations during the time-update and observation-update using the Jacobian of $h$ and $f$.

![Screenshot 2024-11-07 at 1.38.39 PM.png](../../images/Screenshot_2024-11-07_at_1.38.39_PM.png)

The standard EKF functions consist of a time-update and observation-update which simplifies are prior time-update and observation-update equations into Gaussian distributions which can be calculated using closed-form equations (with a long derivation) from EKF.

The whole reason we use Gaussians to model the noise is so that this simplification is possible which makes calculation computationally tractable.

The primary weakness of EKF is that they assume the distributions are linear and Gaussian, which is a weakness in many scenarios.

EKF-SLAM converges as the determinant of the covariance matrix and landmark pair sub-matrices converges toward zero.

> The observation update step requires that all landmarks and
> the joint covariance matrix be updated every time an observation is made. Naively, this means computation grows quadratically with the number of landmarks.

> The standard formulation of the EKF-SLAM solution is especially fragile to incorrect association of observations to landmarks.

Attributing observations to the correct landmarks is challenging.

> EKF-SLAM employs linearized models of nonlinear motion and observation models and so inherits many caveats. Nonlinearity can be a significant problem in EKF-SLAM and leads to inevitable, and sometimes dramatic, inconsistency in solutions.

**2. FastSLAM: Rao-Blackwellized Filter**

> FastSLAM, with its basis in recursive Monte Carlo sampling, or particle filtering, was the first to directly represent the nonlinear process model and non-Gaussian pose distribution.

> The high dimensional state-space of the SLAM problem makes direct application of particle filters computationally infeasible.

[…]

> This is a key property of FastSLAM and the reason for its speed; the map is represented as a set of independent Gaussians, with linear complexity, rather than a joint map covariance with quadratic complexity.

[…]

> Updating the map, for a given trajectory particle $X_{0:k}^{(i)}$, is trivial. Each observed landmark is processed individually as an EKF measurement update from a known pose.

[…]

> Statistically, FastSLAM suffers degeneration due to its inability to forget the past.

### Implementation of SLAM

![Screenshot 2024-11-07 at 12.44.30 PM.png](../../images/Screenshot_2024-11-07_at_12.44.30_PM.png)

> The “explore and return’’ experiment by Newman et al. was a moderate-scale indoor implementation.

> The experiment is remarkable because its return trip was fully autonomous.

![Screenshot 2024-11-07 at 12.44.44 PM.png](../../images/Screenshot_2024-11-07_at_12.44.44_PM.png)

> Guivant and Nebot pioneered the application of SLAM in very large outdoor environments. They addressed computational issues of
> real-time operation, while also dealing with high-speed vehicle motion, non-flat terrain, and dynamic clutter.

### Conclusion

> This article has described the SLAM problem and the essential methods for solving the SLAM problem and has summarized key implementations and demonstrations of the method.